<a href="https://colab.research.google.com/github/DavidJ2209/CodigosUni/blob/main/Mapa%20Tur%C3%ADstico%20Interactivo%20con%20Folium%20y%20Ruta%20OSRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# =================================================================
# MAPA TURÍSTICO - BARRANQUILLA
# =================================================================

!pip -q install folium requests

import folium
import requests
from folium.plugins import MarkerCluster, MeasureControl, Fullscreen, MiniMap, MousePosition

# 2. Definición de lugares
lugares = [
    {"nombre": "Mi Casa (Cra 15 #61)", "cat": "Hogar", "lat": 10.9620913, "lon": -74.8069905, "desc": "Punto de partida en el suroccidente."},
    {"nombre": "Restaurante Los Helechos", "cat": "Gastronomía", "lat": 11.0118195, "lon": -74.8096825, "desc": "Restaurante recomendado de comida en la ciudad."},
    {"nombre": "Restaurante La Matriarca", "cat": "Gastronomía", "lat": 11.0045817, "lon": -74.8119522, "desc": "Restaurante de comida tipica en la ciudad."},
    {"nombre": "Gran Malecón del Río", "cat": "Paseo", "lat": 11.018391, "lon": -74.7937538, "desc": "Paseo a orillas del río Magdalena."},
    {"nombre": "Ventana al Mundo", "cat": "Paseo", "lat": 11.0330772, "lon": -74.8316687, "desc": "Monumento multicolor icónico."},
    {"nombre": "Barrio El Prado", "cat": "Historia", "lat": 10.9983209, "lon": -74.7979901, "desc": "Arquitectura patrimonial histórica."},
    {"nombre": "La Aduana", "cat": "Cultura", "lat": 10.9884957, "lon": -74.7792811, "desc": "Centro cultural y biblioteca histórica."},
    {"nombre": "Zoológico de Barranquilla", "cat": "Naturaleza", "lat": 11.0108114, "lon": -74.7982058, "desc": "Fauna caribeña y educación ambiental."},
    {"nombre": "Estadio Metropolitano", "cat": "Deporte", "lat": 10.9268376, "lon": -74.8015673, "desc": "Casa de la Selección Colombia."},
    {"nombre": "Ciénaga de Mallorquín", "cat": "Naturaleza", "lat": 11.0464097, "lon": -74.8597437, "desc": "Ecoparque y avistamiento de aves."},
    {"nombre": "Catedral Metropolitana", "cat": "Historia", "lat": 10.9881461, "lon": -74.7908534, "desc": "Catedral modernista en Plaza de la Paz."},
    {"nombre": "Plaza de la Paz", "cat": "Historia", "lat": 10.98800, "lon": -74.78916, "desc": "Punto de encuentro masivo."},
    {"nombre": "Aleta del Tiburón", "cat": "Paseo", "lat": 10.9983008, "lon": -74.7729547, "desc": "Monumento al Junior de Barranquilla."},
    {"nombre": "Parque Sagrado Corazón", "cat": "Naturaleza", "lat": 10.9933787, "lon": -74.8184976, "desc": "Pulmón verde con senderos."},
    {"nombre": "Museo del Carnaval", "cat": "Cultura", "lat": 10.9927182, "lon": -74.7876976, "desc": "Memoria de la fiesta más grande."}
]

# Configuración de iconos
icono_cat = {
    "Hogar": ("orange", "home"),
    "Gastronomía": ("red", "cutlery"),
    "Cultura": ("blue", "info-sign"),
    "Paseo": ("green", "cloud"),
    "Naturaleza": ("darkgreen", "tree"),
    "Historia": ("purple", "star"),
    "Deporte": ("cadetblue", "flag")
}

# 3. Inicialización del Mapa
m = folium.Map(location=[10.98, -74.80], zoom_start=13, tiles=None)

folium.TileLayer("OpenStreetMap", name="OSM").add_to(m)
folium.TileLayer("CartoDB positron", name="Claro").add_to(m)
folium.TileLayer("CartoDB dark_matter", name="Oscuro").add_to(m)

# 4. Plugins
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)
MousePosition().add_to(m)
m.add_child(MeasureControl(primary_length_unit="kilometers"))

# 5. Capas y Marcadores
clusters = {}
feature_groups = {}

for p in lugares:
    cat = p["cat"]
    if cat not in feature_groups:
        fg = folium.FeatureGroup(name=f"{cat}", show=True)
        feature_groups[cat] = fg
        clusters[cat] = MarkerCluster().add_to(fg)
        fg.add_to(m)

    color, icon = icono_cat.get(cat, ("gray", "info-sign"))
    folium.Marker(
        location=[p["lat"], p["lon"]],
        tooltip=p["nombre"],
        popup=f"<b>{p['nombre']}</b><br>{p['desc']}",
        icon=folium.Icon(color=color, icon=icon)
    ).add_to(clusters[cat])

# 6. Función y Trazado de Ruta OSRM
def trazar_ruta_osrm(origen, destino, profile="driving"):
    url = f"https://router.project-osrm.org/route/v1/{profile}/{origen[1]},{origen[0]};{destino[1]},{destino[0]}?overview=full&geometries=geojson"
    r = requests.get(url, timeout=30)
    data = r.json()
    ruta = data["routes"][0]
    coords_lonlat = ruta["geometry"]["coordinates"]
    coords_latlon = [(lat, lon) for lon, lat in coords_lonlat]
    return coords_latlon, ruta["distance"]/1000.0, ruta["duration"]/60.0

# Coordenadas de los puntos de la ruta
punto_BarrioPrado = (10.9983209, -74.7979901)
punto_LaMatriarca = (11.0045817, -74.8119522)
punto_malecon = (11.018391, -74.7937538)

ruta_fg = folium.FeatureGroup(name="Ruta Turística", show=True).add_to(m)
tramos = [(punto_BarrioPrado, punto_LaMatriarca), (punto_LaMatriarca, punto_malecon)]
total_km, total_min = 0.0, 0.0

for i, (o, d) in enumerate(tramos, start=1):
    coords, km, minutes = trazar_ruta_osrm(o, d)
    total_km += km
    total_min += minutes
    folium.PolyLine(coords, weight=6, opacity=0.8, color="#e67e22", tooltip=f"Tramo {i}").add_to(ruta_fg)

# --- AQUÍ ESTÁ EL ARREGLO DE LA VENTANITA ---
panel_html = f"""
<div style="background:white; border-radius:10px; padding:10px; border:2px solid #e67e22; width:180px; font-family:sans-serif; box-shadow: 2px 2px 10px rgba(0,0,0,0.2);">
    <b style="color:#d35400; font-size:14px;">Resumen de Ruta</b><br>
    <hr style="margin:5px 0; border:0; border-top:1px solid #eee;">
    <div style="font-size:12px;">
        <b>Distancia:</b> {total_km:.2f} km<br>
        <b>Tiempo:</b> {total_min:.1f} min<br>
        <small style="color:#777;">Prado → Matriarca → Malecón</small>
    </div>
</div>
"""

# Añadimos el panel usando DivIcon en el punto intermedio (La Matriarca)
folium.Marker(
    location=punto_LaMatriarca,
    icon=folium.DivIcon(html=panel_html, icon_anchor=(-15, 0)) # Ajustamos la posición para que no tape el marcador
).add_to(ruta_fg)
# --------------------------------------------

m.fit_bounds([(p["lat"], p["lon"]) for p in lugares])
folium.LayerControl(collapsed=False).add_to(m)
m.save("mapa_barranquilla_final.html")
m